In [22]:
from IPython.core.debugger import set_trace
import numpy as np
import sys
import collections

f=open('day15puzzleinput.txt') #not with read because thats probably the whole file
f=open('day15finaltest.txt') #not with read because thats probably the whole file
lines = [line.rstrip('\n') for line in f]

In [23]:
def bfs(cave, start,targets):
    wall = ["#", 'E','G']
    queue = collections.deque([[start]])
    seen = set([start])
    paths=[]
    while queue:
        #print ('queue',queue)
        path = queue.popleft()
        #print ('queue',queue)
        x, y = path[-1]
        #if (x,y)==(1,4):set_trace() 
        #print (x,y)
        if (x,y) in targets:
            
            #print ('there',(x,y))
            paths.append([(x,y),len(path),path])
            targets.remove((x,y))
            #print ('targets',targets,len(targets))
            
            if len(targets)==0:
                #print ('bla')
                return paths
        
        for x2, y2 in ((x+1,y), (x-1,y), (x,y+1), (x,y-1)):
            
            #if 0 <= x2 < cave.shape[1] and 0 <= y2 < cave.shape[0] and cave[y2][x2] not in wall and (x2, y2) not in seen:
            if 0 <= x2 < cave.shape[0] and 0 <= y2 < cave.shape[1] and cave[(x2,y2)] not in wall and (x2, y2) not in seen:
                queue.append(path + [(x2, y2)])
                #if (x2,y2)==(4,1):set_trace()
                seen.add((x2, y2))
               
    return paths            

In [24]:
  
def select_move(unit):
    
    targets = [enemy[2] for enemy in unitlist if enemy[1]!=unit[1]]
    
    #print ('location',unit[2],'targets',targets)
    #print ('n',neighbors(targets))
    
 
    if (unit[2] in all_neighbors(targets)):
        return False
    paths =  (bfs(cave,unit[2],neighbors(targets)))
    #print ('paths',paths)
    if len(paths)==0:
        return (False)
    else:
        #there is a destination!!
        paths.sort(key=lambda x:(x[1],x[0][0],x[0][1]))
        #print ('final paths',paths)
        destination = paths[0][2][-1]
        #print ('destination',destination)
        adjectent_squares=[]
        for adjecent in neighbors([unit[2]]):
            #print ('adj bfs',adjecent,bfs(cave,adjecent,[destination]))
            path_adj_to_dest = [adjecent,bfs(cave,adjecent,[destination])]
          
        
            if len(path_adj_to_dest[1])!=0:
                adjectent_squares.append(path_adj_to_dest)
        adjectent_squares.sort(key=lambda x:(x[1][0][1],x[0][0],x[0][1]))
        #print('adj sorted',adjectent_squares)
        #print('selected',adjectent_squares[0][0])
        return adjectent_squares[0][0]
   
   
    #select closest target
def all_neighbors(coordinates):
    neighborlist = []
    for x in coordinates:
        neighbors = x + np.array([(0,1),(0,-1),(1,0),(-1,0)])
        
        for x in neighbors:
            neighborlist.append(x)
    #neighbors[np.any(neighbors>0,1)]
    #set_trace()
    #print (neighborlist)
    return [tuple(x) for x in neighborlist if (cave[tuple(x)]!='#')]

def neighbors(coordinates):
    neighborlist = []
    for x in coordinates:
        neighbors = x + np.array([(0,1),(0,-1),(1,0),(-1,0)])
        
        for x in neighbors:
            neighborlist.append(x)
    #neighbors[np.any(neighbors>0,1)]
    #set_trace()
    #print (neighborlist)
    return [tuple(x) for x in neighborlist if (cave[tuple(x)]=='.')]
    
def move(unit):
    #set_trace()
    location = select_move(unit)
    if location != False:
        cave[unit[2]]='.'
        unit[2]=location
        cave[unit[2]]=unit[1]
    return unit
    #attack(unit)
    
    #checkkill()

In [25]:
def attack(unit):
    targets = [enemy[2] for enemy in unitlist if enemy[1]!=unit[1]]
    #print (targets)
    neighbors = all_neighbors([unit[2]])
    #print (neighbors)
    real_targets = [n for n in neighbors if n in targets]
    if len(real_targets)==0: return 'no'
    #print ('real_target', real_targets)
    target_spec=[]
    for i,x in enumerate(unitlist):
        if x[2] in real_targets:
            target_spec.append([i,x])
    target_spec.sort(key=lambda x:(x[1][3],x[1][2][0],x[1][2][1]))
    #print ('final target',target_spec[0])
    return (target_spec[0][0])



In [27]:
def checkend():
    g=[u for u in unitlist if u[1]=='G']
    e=[u for u in unitlist if u[1]=='E'] 
    if (len(g)==0) or (len(e))==0:
        print ('battle-end')
        return True
    else: return False


def fullturn():
    unitlist.sort(key=lambda x:(x[2][0],x[2][1]))
    for unit in unitlist[:]:
        if unit[3]>0:
            #print ('turn of next unit',unit)
            unit = move(unit)


            attacking = attack(unit)

            if attacking != 'no':
                if unitlist[attacking][1]=='E':
                    unitlist[attacking][3]+=-3
                if unitlist[attacking][1]=='G':
                    unitlist[attacking][3]+=-elfpower
                if unitlist[attacking][3]<=0:

                    cave[unitlist[attacking][2]]='.'
                    del unitlist[attacking]
                    if checkend(): return True
            #print (unitlist)
    return False
      
def gameon():
    for turn in range(10000):

        if turn %20 ==0: print (turn)
      
        if fullturn():
            print (turn)
            print ([u[3] for u in unitlist])
            print (sum([u[3] for u in unitlist])*(turn))
            print (sum([u[3] for u in unitlist])*(turn+1))
            print ('elves alive',len(unitlist))
            print (unitlist)
            return
        else:
            print (unitlist)
            

            
def initiate():            
    global cave
    cave = np.chararray(unicode=True,shape=(len(lines),len(lines[0])))
    for x,line in enumerate(lines):
        #print ((line))
        cave[x]=list(line)

    global unitlist 
    unitlist = []
    number= 0

    for cell, value in np.ndenumerate(cave):

        if (value == 'G') or (value == 'E'):

            unitlist.append([number,value,cell,200])

            number+=1
           

for x in range (3,4):
    initiate()
    
    elfpower = x
    print ('elfpower',x)
    gameon()

elfpower 3
0
[[0, np.str_('G'), (np.int64(1), np.int64(2)), 197], [1, np.str_('E'), (2, 2), 197], [2, np.str_('G'), (np.int64(2), np.int64(7)), 200], [3, np.str_('G'), (np.int64(5), np.int64(2)), 200], [4, np.str_('G'), (np.int64(6), np.int64(5)), 200], [5, np.str_('G'), (np.int64(7), np.int64(5)), 200]]
[[0, np.str_('G'), (np.int64(1), np.int64(2)), 194], [1, np.str_('E'), (2, 2), 194], [2, np.str_('G'), (np.int64(1), np.int64(7)), 200], [3, np.str_('G'), (np.int64(4), np.int64(2)), 200], [4, np.str_('G'), (np.int64(6), np.int64(4)), 200], [5, np.str_('G'), (np.int64(7), np.int64(4)), 200]]
[[0, np.str_('G'), (np.int64(1), np.int64(2)), 191], [2, np.str_('G'), (np.int64(1), np.int64(6)), 200], [1, np.str_('E'), (2, 2), 188], [3, np.str_('G'), (np.int64(3), np.int64(2)), 200], [4, np.str_('G'), (np.int64(6), np.int64(3)), 200], [5, np.str_('G'), (np.int64(7), np.int64(3)), 200]]
[[0, np.str_('G'), (np.int64(1), np.int64(2)), 188], [2, np.str_('G'), (np.int64(1), np.int64(5)), 200], [1,

C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_15408\154614937.py:53: DeprecationWarning: `np.chararray` is deprecated and will be removed from the main namespace in the future. Use an array with a string or bytes dtype instead.
  cave = np.chararray(unicode=True,shape=(len(lines),len(lines[0])))


In [1]:
732+36+37+38+39+40+41+42

1005

In [9]:
[unit for unit in unitlist if unit[1]!='G']

[[4, 'E', (4, 4), 200]]

In [22]:
a = [[(7, 6), [[(3, 5), 6, [(7, 6), (6, 6), (5, 6), (5, 5), (4, 5), (3, 5)]]]], [(6, 7), [[(3, 5), 6, [(6, 7), (5, 7), (4, 7), (3, 7), (3, 6), (3, 5)]]]]]
sort(key=lambda x:(x[1][0][1],x[1][0][0][0],x[1][0][0][1]))

In [28]:
215302 too high

7

In [67]:
a=[1,'b',3]
a.find('b')

AttributeError: 'list' object has no attribute 'find'

[array([8, 7]), array([7, 8])]

In [65]:
def get_neighbours(p, exclude_p=True, shape=None):

    ndim = len(p)

    offsets = np.array([[1,0],[0,1],[-1,0],[0,-1]])

    neighbours = p + offsets    # apply offsets to p

    # optional: exclude out-of-bounds indices
    if shape is not None:
        valid = np.all((neighbours < np.array(shape)) & (neighbours >= 0), axis=1)
        neighbours = neighbours[valid]

    return neighbours


get_neighbours((7,7),shape=cave.shape)
#for coor, input in np.ndenumerate(cave):
#    print get_neighbors

array([[8, 7],
       [7, 8],
       [6, 7],
       [7, 6]])

In [ ]:
def

In [71]:
#print (np.argwhere(cave=='G'))
for cell, value in np.ndenumerate(cave):
    print (cell,value)
cave[cave=='G']

[[1 1]
 [1 4]
 [1 7]
 [4 1]
 [4 7]
 [7 1]
 [7 4]
 [7 7]]
(0, 0) #
(0, 1) #
(0, 2) #
(0, 3) #
(0, 4) #
(0, 5) #
(0, 6) #
(0, 7) #
(0, 8) #
(1, 0) #
(1, 1) G
(1, 2) .
(1, 3) .
(1, 4) G
(1, 5) .
(1, 6) .
(1, 7) G
(1, 8) #
(2, 0) #
(2, 1) .
(2, 2) .
(2, 3) .
(2, 4) .
(2, 5) .
(2, 6) .
(2, 7) .
(2, 8) #
(3, 0) #
(3, 1) .
(3, 2) .
(3, 3) .
(3, 4) .
(3, 5) .
(3, 6) .
(3, 7) .
(3, 8) #
(4, 0) #
(4, 1) G
(4, 2) .
(4, 3) .
(4, 4) E
(4, 5) .
(4, 6) .
(4, 7) G
(4, 8) #
(5, 0) #
(5, 1) .
(5, 2) .
(5, 3) .
(5, 4) .
(5, 5) .
(5, 6) .
(5, 7) .
(5, 8) #
(6, 0) #
(6, 1) .
(6, 2) .
(6, 3) .
(6, 4) .
(6, 5) .
(6, 6) .
(6, 7) .
(6, 8) #
(7, 0) #
(7, 1) G
(7, 2) .
(7, 3) .
(7, 4) G
(7, 5) .
(7, 6) .
(7, 7) G
(7, 8) #
(8, 0) #
(8, 1) #
(8, 2) #
(8, 3) #
(8, 4) #
(8, 5) #
(8, 6) #
(8, 7) #
(8, 8) #


chararray(['G', 'G', 'G', 'G', 'G', 'G', 'G', 'G'],
          dtype='<U1')

In [19]:
offset_idx

array([[0, 0],
       [0, 1],
       [0, 2],
       [1, 0],
       [1, 1],
       [1, 2],
       [2, 0],
       [2, 1],
       [2, 2]])

np.indices((3,) * 2)

In [39]:
(0,1)+(2,2)

(0, 1, 2, 2)

In [17]:
import numpy as np
c=(6,6) + np.array([(0,1),(0,-1),(1,0),(-1,0)])

In [20]:
cave[tuple(c[0])]

'.'

In [ ]:
from IPython.core.debugger import set_trace
a=1
b=2
def add_to_life_universe_everything(x):
    c=3
    answer = [42,98]
    set_trace()
    answer += x
    
    return answer

add_to_life_universe_everything(12)

> <ipython-input-5-e93a9c2e83b5>(8)add_to_life_universe_everything()
      6     answer = [42,98]
      7     set_trace()
----> 8     answer += x
      9 
     10     return answer

ipdb> answer
[42, 98]
ipdb> answer[0]
42


In [2]:
def add_to_life_universe_everything(x):
    answer = 42
    import pdb; pdb.set_trace()
    answer += x
    
    return answer

add_to_life_universe_everything(12)

> <ipython-input-2-b567e28d9592>(4)add_to_life_universe_everything()
-> answer += x
(Pdb) answer
42
(Pdb) >
*** SyntaxError: invalid syntax
(Pdb) n
> <ipython-input-2-b567e28d9592>(6)add_to_life_universe_everything()
-> return answer
(Pdb) answer
54
(Pdb) n
--Return--
> <ipython-input-2-b567e28d9592>(6)add_to_life_universe_everything()->54
-> return answer
(Pdb) answer
54
(Pdb) c


54